# Базы данных (Database) и СУБД

* Долговременное хранение данных
* Хранение связанных данных и отношенийй между ними, систематизация данных
* Эффективный поиск и селекция

## СУБД

* Доступ к данных из нескольких программ
* Обеспечение атомарности, согласованности, изолированности и долговечности данных (ACID)

## ACID
* Атомарность - гарантирует, что никакая транзакция не будет зафиксирована в системе частично.
* Согласованность - после заверешения транзакции данные целостны и не противоречат друг другу.
* Изолированность - во время выполнения транзакции параллельные транзакции не должны оказывать влияния на её результат.
* Долговечность - если транзакция успешно проведена, то значит данные действительно сохранены.

**SQL** - распространненный язык для составления запросов к БД

## Python Database API Specification v2.0

* [PEP 249](https://www.python.org/dev/peps/pep-0249/)
* Приведение модулей для работы с различным базами данных к единому интерфейсу.
* Обеспечивает универсальность работы и модульную замену
* [Python Database Propgramming](https://wiki.python.org/moin/DatabaseProgramming)

## SQLite

* Компактная, встраиваемая БД.
* Хранит данные в файле, блокирующая однопользовательская
* [Сайте](https://www.sqlite.org/index.html)
* [Wiki](https://ru.wikipedia.org/wiki/SQLite)

In [ ]:
import sqlite3

Можно подключится к файлу БД

In [ ]:
# con = sqlite3.connect('example.db') 

Mожно создать БД в памяти, см. [In-memory database](https://en.wikipedia.org/wiki/In-memory_database)

In [ ]:
con = sqlite3.connect(':memory:')

In [ ]:
# TODO(Придумать схему таблицы и код для генерации данных)
# TODO(Использование адаптеров и конверторов)

In [ ]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()


In [ ]:
# Successful, con.commit() is called automatically afterwards
with con:
    con.execute("", ("",))

In [ ]:
# Connection object used as context manager only commits or rollbacks transactions,
# so the connection object should be closed manually
con.close()

## SQAlchemy

* Высокоуровневая библиотека для работы с базами данными
* Для доступа в базу данных может использовать разные драйверы удовлетворяющий DPAPI

![](./sqla_engine_arch.png)

* **SQLAlchemy Core** --- Python DSL for SQL
* **SQLAlchemy ORM** --- object relational mapping

### Object Relational Mapping

Проблема:
 * Данные в БД хранятся в виде таблиц
 * Данные в программе представлены в виде классов и объектов
 * Возникает необходимость писать лишний код для взаимодейсвтия с базой данной --- CRUD операции: (Create, Read, Update, Delete)
Решение:
 * ORM технологии для автоматического отображения классов в базу данных

In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [ ]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, DateTime
import datetime

metadata = MetaData()
table = Table('table', metadata,
    Column('id', Integer(), primary_key=True),
    Column('name', String(200), nullable=False),
    Column('timestamp', DateTime(), default=datetime.datetime.now),
)

metadata.create_all(engine)

In [17]:
stmt = table.insert().values(name="NAMENAME")
print(stmt)

INSERT INTO "table" (name, timestamp) VALUES (:name, :timestamp)


In [18]:
with engine.connect() as conn:
    r = conn.execute(stmt)
    print(r.inserted_primary_key)

2021-11-09 15:41:53,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 15:41:53,732 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 15:41:53,733 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ('NAMENAME', '2021-11-09 15:41:53.731127')
(1,)
2021-11-09 15:41:53,734 INFO sqlalchemy.engine.Engine ROLLBACK


In [23]:
metadata.drop_all(engine)

2021-11-09 15:46:16,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 15:46:16,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("table")
2021-11-09 15:46:16,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 15:46:16,353 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("table")
2021-11-09 15:46:16,354 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 15:46:16,355 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class MySomeTable(Base):
    __tablename__ = "table"
    id = Column(Integer, primary_key=True)
    name = Column( String(200), nullable=False)
    timestamp = Column(DateTime, default=datetime.datetime.now)

Base.metadata.create_all(engine)

2021-11-09 15:49:18,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 15:49:18,290 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("table")
2021-11-09 15:49:18,291 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 15:49:18,292 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("table")
2021-11-09 15:49:18,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 15:49:18,294 INFO sqlalchemy.engine.Engine 
CREATE TABLE "table" (
	id INTEGER NOT NULL, 
	name VARCHAR(200) NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2021-11-09 15:49:18,294 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2021-11-09 15:49:18,297 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy.orm import mapper

class MyMapperTable(object):
    pass

mapper(MyMapperTable, table)

In [29]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

data1 = MySomeTable(name="SOMENAME")
data2 = MySomeTable(name="ANOTHERNAME")

session.add(data1)
session.add(data2)
print(session.new)
session.commit()

IdentitySet([<__main__.MySomeTable object at 0x7fddf12143d0>, <__main__.MySomeTable object at 0x7fddf12140a0>])
2021-11-09 15:49:21,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 15:49:21,499 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 15:49:21,500 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ('SOMENAME', '2021-11-09 15:49:21.499356')
2021-11-09 15:49:21,503 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 15:49:21,504 INFO sqlalchemy.engine.Engine [cached since 0.005681s ago] ('ANOTHERNAME', '2021-11-09 15:49:21.502975')
2021-11-09 15:49:21,506 INFO sqlalchemy.engine.Engine COMMIT
